<a href="https://colab.research.google.com/github/ddhackiisc/code/blob/master/DILI/DILI_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import padelpy

ModuleNotFoundError: ignored

In [2]:
import pandas as pd
import numpy as np
import pybel
from rdkit import Chem


ModuleNotFoundError: ignored

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
train_df = pd.read_csv("combined_test.csv")
train_smiles = train_df['smiles'].to_numpy()
train_labels = train_df['label'].to_numpy()



In [ ]:
f = from_smiles(train_smiles[0], descriptors = False, fingerprints = True)